In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import warnings
from IPython.display import display, HTML, Markdown
import re
import time
import pandas as pd
import json
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j")
import logging
load_dotenv()

True

# Initialization Embeddings model

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Connect to Neo4j Database

In [3]:
NEO4j_URI = 'bolt://localhost:7687'
NEO4j_USER = 'neo4j'
NEO4j_PASSWORD = os.environ.get('pass')
graph = Neo4jGraph(url=NEO4j_URI, username=NEO4j_USER, password=NEO4j_PASSWORD)

# Hybrid Searching (Semantic + graph)

In [4]:
def retrieve_similar_events(query_text, graph, embeddings, top_k=5, embedding_weight=0.6, structure_weight=0.4, similarity_threshold=0.5):
    """
    Retrieve similar events using a hybrid approach with optimized query performance.
    """
    query_embedding = embeddings.embed_query(query_text)
    
    cypher_query = """
    MATCH (e:Event)
    WHERE e.embedding IS NOT NULL AND e.text IS NOT NULL
    
    // Efficient connection counting using COUNT
    OPTIONAL MATCH (e)-[:RESULTS_IN]->(effect:Effect)
    WITH e, COUNT(effect) AS effect_count
    
    OPTIONAL MATCH (e)<-[:CAUSES]-(cause:Cause)
    WITH e, effect_count, COUNT(cause) AS cause_count
    
    OPTIONAL MATCH (e)-[:HAS_TRIGGER]->(trigger:Trigger)
    WITH e, effect_count, cause_count, COUNT(trigger) AS trigger_count
    
    // Calculate structural score (maintain original binary approach)
    WITH e, effect_count, cause_count, trigger_count,
         CASE 
             WHEN effect_count + cause_count + trigger_count = 0 
             THEN 0.0 
             ELSE 1.0 
         END AS structural_score
    
    // Calculate similarity and hybrid score
    WITH e, effect_count, cause_count, trigger_count, structural_score,
         gds.similarity.cosine(e.embedding, $query_embedding) AS embedding_similarity,
         ($embedding_weight * gds.similarity.cosine(e.embedding, $query_embedding) + $structure_weight * structural_score) AS hybrid_score
    
    WHERE hybrid_score >= $similarity_threshold
    
    // Late text collection for final candidates
    CALL {
        WITH e
        OPTIONAL MATCH (e)-[:RESULTS_IN]->(effect:Effect)
        RETURN COLLECT(effect.text) AS effect_texts
    }
    CALL {
        WITH e
        OPTIONAL MATCH (e)<-[:CAUSES]-(cause:Cause)
        RETURN COLLECT(cause.text) AS cause_texts
    }
    CALL {
        WITH e
        OPTIONAL MATCH (e)-[:HAS_TRIGGER]->(trigger:Trigger)
        RETURN COLLECT(trigger.text) AS trigger_texts
    }
    
    RETURN e.text AS text,
           e.id AS event_id,
           hybrid_score,
           embedding_similarity,
           structural_score,
           effect_count,
           cause_count,
           trigger_count,
           effect_texts,
           cause_texts,
           trigger_texts
    ORDER BY hybrid_score DESC
    LIMIT $top_k
    """
    
    try:
        logging.getLogger("neo4j").setLevel(logging.ERROR)
        results = graph.query(
            cypher_query,
            {
                "query_embedding": query_embedding,
                "embedding_weight": float(embedding_weight),
                "structure_weight": float(structure_weight),
                "similarity_threshold": float(similarity_threshold),
                "top_k": int(top_k)
            }
        )
        
        similar_events = []
        for record in results:
            if record['text']:
                similar_events.append({
                    'text': record['text'],
                    'event_id': record.get('event_id', 'N/A'),
                    'hybrid_score': round(float(record['hybrid_score']), 3),
                    'embedding_similarity': round(float(record['embedding_similarity']), 3),
                    'structural_score': round(float(record['structural_score']), 3),
                    'connection_counts': {
                        'effects': int(record['effect_count']),
                        'causes': int(record['cause_count']),
                        'triggers': int(record['trigger_count'])
                    },
                    'connected_nodes': {
                        'causes': [str(cause) for cause in record['cause_texts'] if cause],
                        'effects': [str(effect) for effect in record['effect_texts'] if effect],
                        'triggers': [str(trigger) for trigger in record['trigger_texts'] if trigger]
                    }
                })
        
        return similar_events
    
    except Exception as e:
        print(f"Error during query execution: {str(e)}")
        return []

# Testing out Hybrid Query 

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
query_example = "I observed the attack on the police , I have no doubt about it , '' Modiba said during cross-examination ."
similar_events = retrieve_similar_events(
    query_example,
    graph,
    embeddings,
    top_k=5,
    embedding_weight=0.6,
    structure_weight=0.4,
    similarity_threshold=0.5
)

print(f"\nFound {len(similar_events)} similar events:")
for i, event in enumerate(similar_events, 1):
    print(f"\n{i}. Event: {event['text']}")
    print(f"   ID: {event['event_id']}")
    print(f"   Hybrid Score: {event['hybrid_score']}")
    print(f"   Embedding Similarity: {event['embedding_similarity']}")
    print(f"   Structural Score: {event['structural_score']}")
    
    # Print connected nodes in the requested format
    print("   Connected Nodes:")
    print("   Causes:")
    for cause in event['connected_nodes']['causes']:
        print(f"   - {cause}")
    print("   Effects:")
    for effect in event['connected_nodes']['effects']:
        print(f"   - {effect}")
    print("   Triggers:")
    for trigger in event['connected_nodes']['triggers']:
        print(f"   - {trigger}")
    print("   ---")


Found 5 similar events:

1. Event: i observed the attack on the police , i have no doubt about it , '' modiba said during cross-examination .
   ID: event_20
   Hybrid Score: 1.0
   Embedding Similarity: 1.0
   Structural Score: 1.0
   Connected Nodes:
   Causes:
   - Modiba said
   Effects:
   - that he observed the attack on the police and had no doubt about it
   Triggers:
   - during cross-examination
   ---

2. Event: thanks to the providential arrest of a terrorist in mumbai and his subsequent interrogation , the police are asserting with a considerable degree of confidence that the let planned and orchestrated the attacks that took the lives of more than 180 people .
   ID: event_1131
   Hybrid Score: 0.757
   Embedding Similarity: 0.595
   Structural Score: 1.0
   Connected Nodes:
   Causes:
   - Thanks to the providential arrest of a terrorist in Mumbai and his subsequent interrogation
   Effects:
   - the police are asserting with a considerable degree of confidence that the

In [6]:
def generate(llm, input, graph, embeddings):
    prompt="""
    <task>
        You are an advanced language model tasked with identifying and tagging explicit cause-and-effect relationships in complex sentences. 
        <rule>Do not modify the original text when adding tags.</rule>
    </task>

    <VERY IMPORTANT RULE>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present). If you miss this in the end the answer will be considered wrong</VERY IMPORTANT RULE>

    <steps>
        <step number="1">
            <title>Causality Determination</title>
            <description>
                <point>Analyze the input sentence to determine if it contains an explicit causal relationship.</point>
                <point>If the sentence is <b>not causal</b>, output <b>NoTag</b> and set the label to <b>0</b>. <rule>Do not perform any tagging.</rule></point>
                <point>If the sentence is causal, proceed only if <b>all three elements</b>—cause, trigger, and effect—are present.</point>
                <point>If any element is missing, output <b>NoTag</b> and set the label to <b>0</b>.</point>
            </description>
        </step>

        <step number="2">
            <title>Tagging Instructions</title>
            <tags>
                <tag>
                    <name>Cause</name>
                    <format>&lt;cause&gt;...&lt;/cause&gt;</format>
                    <definition>The event, action, or condition that leads to an outcome.</definition>
                    <question>What caused this?</question>
                </tag>
                <tag>
                    <name>Trigger</name>
                    <format>&lt;trigger&gt;...&lt;/trigger&gt;</format>
                    <definition>The word or phrase that explicitly indicates causality (e.g., because, due to).</definition>
                </tag>
                <tag>
                    <name>Effect</name>
                    <format>&lt;effect&gt;...&lt;/effect&gt;</format>
                    <definition>The outcome or result of the cause.</definition>
                    <question>What is the result of this?</question>
                </tag>
            </tags>
        </step>

        <step number="3">
            <title>Causality Tests</title>
            <description>
                <point>The effect should answer "Why?" (i.e., it must clearly result from the cause).</point>
                <point>The cause must precede the effect in time.</point>
                <point>The effect should not occur without the cause.</point>
                <point>The cause and effect cannot be swapped without changing the meaning.</point>
                <point>The sentence can be rephrased as “X causes Y” or “Due to X, Y.”</point>
            </description>
        </step>

        <step number="4">
            <title>Additional Guidelines</title>
            <guidelines>
                <point>The sentence must have explicit cause-effect wording.</point>
                <point>If there are multiple causes or effects, tag each instance separately.</point>
                <point>Do not alter the original sentence structure when adding tags.</point>
                <point>The final output must include all three tags: <b>&lt;cause&gt;</b>, <b>&lt;trigger&gt;</b>, and <b>&lt;effect&gt;</b>.</point>
                <point>If any tag is missing, consider the sentence non-causal.</point>
                <point>If no explicit causal relationship is identified, output <b>NoTag</b>.</point>
            </guidelines>
        </step>

        <step number="5">
            <title>Output Format</title>
            <output>
                <format>Return a JSON object with two keys: "tagged" and "label".</format>
                <key name="tagged">Contains the original sentence with all tags added.</key>
                <key name="label">Set to <b>1</b> if a causal relationship is successfully tagged; otherwise, <b>0</b>.</key>
            </output>
        </step>
    </steps>

    <examples>
        <example>
            <input>Biswal, who had travelled to the city from Washington to speak at the Indian Consulate General's Media-India Lecture Series yesterday, said not only were there a large number of Indian victims in the attack but there were Americans also who lost their lives.</input>
            <tagged>Biswal, who had <trigger>travelled to the city from Washington</trigger> to <cause>speak at the Indian Consulate General's Media-India Lecture Series</cause> yesterday, said <effect>not only were there a large number of Indian victims in the attack</effect> but <effect>there were Americans also who lost their lives</effect>.</tagged>
            <label>1</label>
        </example>

        <more examples>
        {similar_examples}
        </more examples>

        <example>
            <input>The African nationals had sustained minor injuries in the attack that took place in South Delhi's Mehrauli area on Friday.</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>
        
        <example>
            <input>On Sunday , Swaraj spoke to Rajnath Singh and Delhi Lieutenant Governor Najeeb Jung about the attacks and said she was assured that the culprits would soon be arrested .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>We will continue our strike till we get an assurance from the Government .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>There is no information about the involvement of the NDFB(S) underground group in the blast , he added .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

    </examples>

    <qa>
        <requirement>For sentences that are tagged as causal, include a brief description explaining the cause-and-effect relationship.</requirement>
        <example>
            <sentence>&lt;effect>The participants raised slogans&lt;/effect> &lt;trigger>demanding&lt;/trigger> &lt;cause>stern action to check atrocities against women in the name of moral policing&lt;/cause></sentence>
            <explanation>Why did they raise slogans? To demand action against atrocities.</explanation>
            <label>1</label>
        </example>
    </qa>

    <final_instruction>
        Now, process the following input text:
        <input>{input_text}</input>
        <note>Reconsider your decisions step by step internally (chain-of-thought) but do not output your reasoning.</note>
        <IMPORTANT>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present).</IMPORTANT>
    </final_instruction>
    """
    prompt_template = PromptTemplate(
		input_variables=["similar_examples", "input_text"],
		template=prompt
	)

    llm_chain = LLMChain(
    	llm=llm,
    	prompt=prompt_template
	)

    similar_events = retrieve_similar_events(
        input,
        graph,
        embeddings,
        top_k=20,
        embedding_weight=0.6,
        structure_weight=0.4,
        similarity_threshold=0.5
    )
    
    output = llm_chain.run(
	    similar_examples=similar_events,
	    input_text=input
	)

    return output

In [7]:
llm = ChatGroq(
    groq_api_key=os.environ.get('GROQ_API'),
    model_name='deepseek-r1-distill-llama-70b'
)

In [8]:
query_example = "As commuters rushed past , activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination ."
llm_output = generate(llm, query_example, graph, embeddings)

print(llm_output)

<think>
Alright, let me try to figure out how to approach this problem. So, the task is to identify and tag explicit cause-and-effect relationships in a given sentence. The user has provided a detailed set of instructions, so I need to make sure I follow them carefully.

First, I need to determine if the input sentence is causal. That means checking if there's a clear cause, trigger, and effect present. If any of these are missing, I should output "NoTag" and set the label to 0.

Looking at the input sentence: "As commuters rushed past, activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination."

I'll break it down. The sentence describes two main actions: commuters rushing past and activists waving flags and putting up banners. The activists are urging commuters to join a struggle. So, the cause seems to be the activists' actions—waving flags and erecting banners. The effect is that they're urging othe

# Get predictions with 5 examples retrieved for Test Set

In [2]:
df = pd.read_csv('test_data.csv')

In [13]:
total_rows = len(df)  
start_time = time.time()

with tqdm(total=total_rows, desc="Processing", unit="rows") as pbar:
    for index, row in df.iterrows():

        try:
            with open("test_pred.json", "r", encoding="utf-8") as f:
                data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            data = []

        flag = 0

        for i in range(len(data)):
            if data[i]["original"] == row["text"]:
                print(i, " Skipped.")
                flag = 1
                break

        if flag == 1:
            pbar.update(1)
            continue

        llm_output = generate(llm, row["text"], graph, embeddings)
        match = re.search(r"\{.*\}", llm_output, re.DOTALL)
        if match:
            json_data = match.group(0)
            try:
                json_data = json.loads(json_data)
            except json.JSONDecodeError:
                print(f"Error: Invalid JSON at row {index}. Skipping...")
                pbar.update(1)
                continue
        else:
            print(f"Error: No JSON found in row {index}. Skipping...")
            pbar.update(1)
            continue

        tagged = json_data.get("tagged", "")
        label = json_data.get("label", "")
        data.append({"tagged": tagged, "label": label, "original": row["text"]})

        with open("test_pred.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

        pbar.update(1)
        elapsed_time = time.time() - start_time
        avg_time_per_row = elapsed_time / (index + 1)
        remaining_time = avg_time_per_row * (total_rows - (index + 1))
        pbar.set_postfix({"ETA (sec)": f"{remaining_time:.2f}"})

        print("\nOriginal Sentence:", row["text"])
        print("Tagged Sentence:", tagged)
        print("Label:", label)
        print("---")

Processing:  44%|████▍     | 143/323 [00:00<00:00, 1422.23rows/s]

0  Skipped.
1  Skipped.
2  Skipped.
3  Skipped.
4  Skipped.
5  Skipped.
6  Skipped.
7  Skipped.
8  Skipped.
9  Skipped.
10  Skipped.
11  Skipped.
12  Skipped.
13  Skipped.
14  Skipped.
15  Skipped.
16  Skipped.
17  Skipped.
18  Skipped.
19  Skipped.
20  Skipped.
21  Skipped.
22  Skipped.
23  Skipped.
24  Skipped.
25  Skipped.
26  Skipped.
27  Skipped.
28  Skipped.
29  Skipped.
30  Skipped.
31  Skipped.
32  Skipped.
33  Skipped.
34  Skipped.
35  Skipped.
36  Skipped.
37  Skipped.
38  Skipped.
39  Skipped.
40  Skipped.
41  Skipped.
42  Skipped.
43  Skipped.
44  Skipped.
45  Skipped.
46  Skipped.
47  Skipped.
48  Skipped.
49  Skipped.
50  Skipped.
51  Skipped.
52  Skipped.
53  Skipped.
54  Skipped.
55  Skipped.
56  Skipped.
57  Skipped.
58  Skipped.
59  Skipped.
60  Skipped.
61  Skipped.
62  Skipped.
63  Skipped.
64  Skipped.
65  Skipped.
66  Skipped.
67  Skipped.
68  Skipped.
69  Skipped.
70  Skipped.
71  Skipped.
72  Skipped.
73  Skipped.
74  Skipped.
75  Skipped.
76  Skipped.
77  Skipp

Processing:  70%|██████▉   | 226/323 [00:02<00:00, 1422.23rows/s, ETA (sec)=1.01]


Original Sentence: Twenty activists from the group made a detour and carried the coffin to the Office of the Commissioner for the Ministry of Foreign Affairs in Kennedy Road .
Tagged Sentence: NoTag
Label: 0
---


Processing:  70%|███████   | 227/323 [00:04<00:00, 1422.23rows/s, ETA (sec)=1.91]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|█████████▉| 322/323 [44:18<00:27, 27.12s/rows, ETA (sec)=8.26] 


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , <cause>three explosions in the south of the region</cause> <trigger>killed</trigger> <effect>two people</effect> and <trigger>injured</trigger> <effect>many others</effect> .
Label: 1
---


Processing: 100%|██████████| 323/323 [44:48<00:00,  8.32s/rows, ETA (sec)=0.00]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the clashes are a desperate <effect>protest</effect> <trigger>against</trigger> <cause>religious oppression and economic marginalisation</cause>.
Label: 1
---


# Get F1 Score

In [3]:
with open("test_pred.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [4]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [5]:
df_predictions

,tagged,label,original
0,The movement was <effect>catapulted into the h...,1,The movement was catapulted into the headlines...
1,Several thousand protesters took to the street...,1,Several thousand protesters took to the street...
2,"“They try every means to oppress us,” complain...",1,"“ They try every means to oppress us , ” compl..."
3,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,“ It is somehow like a revolution – no one wil...,1,“ It is somehow like a revolution – no one wil...
...,...,...,...
318,"National Freedom Party spokesman, Sabelo Sigud...",1,"National Freedom Party spokesman , Sabelo Sigu..."
319,NoTag,0,He described the shooting as `` evil '' .
320,"On Sunday , <cause>three explosions in the sou...",1,"On Sunday , three explosions in the south of t..."
321,Uighur groups abroad say the clashes are a des...,1,Uighur groups abroad say the clashes are a des...


In [6]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [7]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [8]:
df_predictions

,tagged_sentence,label,original_sentence
0,The movement was <effect>catapulted into the h...,1,The movement was catapulted into the headlines...
1,Several thousand protesters took to the street...,1,Several thousand protesters took to the street...
2,"“They try every means to oppress us,” complain...",1,"“ They try every means to oppress us , ” compl..."
3,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,“ It is somehow like a revolution – no one wil...,1,“ It is somehow like a revolution – no one wil...
...,...,...,...
318,"National Freedom Party spokesman, Sabelo Sigud...",1,"National Freedom Party spokesman , Sabelo Sigu..."
319,NoTag,0,He described the shooting as `` evil '' .
320,"On Sunday , <cause>three explosions in the sou...",1,"On Sunday , three explosions in the south of t..."
321,Uighur groups abroad say the clashes are a des...,1,Uighur groups abroad say the clashes are a des...


In [9]:
df

,index,text,label,agreement,num_votes,sample_set
0,train_10_0,The movement was catapulted into the headlines...,1,0.666667,6,train_10
1,train_10_1,Several thousand protesters took to the street...,1,0.833333,6,train_10
2,train_10_2,"“ They try every means to oppress us , ” compl...",1,0.833333,6,train_10
3,train_10_3,"Chan Ho-tin , the 25-year-old founder of the H...",0,0.833333,6,train_10
4,train_10_4,“ It is somehow like a revolution – no one wil...,1,0.500000,6,train_10
...,...,...,...,...,...,...
318,train_10_318,"Police spokesman , Lieutenant Colonel Thulani ...",0,0.500000,4,train_10
319,train_10_319,"National Freedom Party spokesman , Sabelo Sigu...",1,0.750000,4,train_10
320,train_10_320,He described the shooting as `` evil '' .,0,1.000000,4,train_10
321,train_10_321,"On Sunday , three explosions in the south of t...",1,1.000000,4,train_10


In [10]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [11]:
df_predictions

,tagged_sentence,label,original_sentence
0,The movement was <effect>catapulted into the h...,1,The movement was catapulted into the headlines...
1,Several thousand protesters took to the street...,1,Several thousand protesters took to the street...
2,"“They try every means to oppress us,” complain...",1,"“ They try every means to oppress us , ” compl..."
3,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,“ It is somehow like a revolution – no one wil...,1,“ It is somehow like a revolution – no one wil...
...,...,...,...
318,"National Freedom Party spokesman, Sabelo Sigud...",1,"National Freedom Party spokesman , Sabelo Sigu..."
319,NoTag,0,He described the shooting as `` evil '' .
320,"On Sunday , <cause>three explosions in the sou...",1,"On Sunday , three explosions in the south of t..."
321,Uighur groups abroad say the clashes are a des...,1,Uighur groups abroad say the clashes are a des...


In [12]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [13]:
common_df

,index,text,label_df,agreement,num_votes,sample_set,tagged_sentence,label_pred,original_sentence
0,train_10_0,The movement was catapulted into the headlines...,1,0.666667,6,train_10,The movement was <effect>catapulted into the h...,1,The movement was catapulted into the headlines...
1,train_10_1,Several thousand protesters took to the street...,1,0.833333,6,train_10,Several thousand protesters took to the street...,1,Several thousand protesters took to the street...
2,train_10_2,"“ They try every means to oppress us , ” compl...",1,0.833333,6,train_10,"“They try every means to oppress us,” complain...",1,"“ They try every means to oppress us , ” compl..."
3,train_10_3,"Chan Ho-tin , the 25-year-old founder of the H...",0,0.833333,6,train_10,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,train_10_4,“ It is somehow like a revolution – no one wil...,1,0.500000,6,train_10,“ It is somehow like a revolution – no one wil...,1,“ It is somehow like a revolution – no one wil...
...,...,...,...,...,...,...,...,...,...
318,train_10_318,"Police spokesman , Lieutenant Colonel Thulani ...",0,0.500000,4,train_10,NoTag,0,"Police spokesman , Lieutenant Colonel Thulani ..."
319,train_10_319,"National Freedom Party spokesman , Sabelo Sigu...",1,0.750000,4,train_10,"National Freedom Party spokesman, Sabelo Sigud...",1,"National Freedom Party spokesman , Sabelo Sigu..."
320,train_10_320,He described the shooting as `` evil '' .,0,1.000000,4,train_10,NoTag,0,He described the shooting as `` evil '' .
321,train_10_321,"On Sunday , three explosions in the south of t...",1,1.000000,4,train_10,"On Sunday , <cause>three explosions in the sou...",1,"On Sunday , three explosions in the south of t..."


In [15]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.7774480712166172
Accuracy: 0.7678018575851393
Precision: 0.8238993710691824
Recall: 0.7359550561797753
Matthews Correlation Coefficient: 0.5400757092117167


# F1 Score calculation after 10 examples given 

In [10]:
total_rows = len(df)  
start_time = time.time()

with tqdm(total=total_rows, desc="Processing", unit="rows") as pbar:
    for index, row in df.iterrows():

        try:
            with open("test_pred_1.json", "r", encoding="utf-8") as f:
                data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            data = []

        flag = 0

        for i in range(len(data)):
            if data[i]["original"] == row["text"]:
                print(i, " Skipped.")
                flag = 1
                break

        if flag == 1:
            pbar.update(1)
            continue

        llm_output = generate(llm, row["text"], graph, embeddings)
        match = re.search(r"\{.*\}", llm_output, re.DOTALL)
        if match:
            json_data = match.group(0)
            try:
                json_data = json.loads(json_data)
            except json.JSONDecodeError:
                print(f"Error: Invalid JSON at row {index}. Skipping...")
                pbar.update(1)
                continue
        else:
            print(f"Error: No JSON found in row {index}. Skipping...")
            pbar.update(1)
            continue

        tagged = json_data.get("tagged", "")
        label = json_data.get("label", "")
        data.append({"tagged": tagged, "label": label, "original": row["text"]})

        with open("test_pred_1.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

        pbar.update(1)
        elapsed_time = time.time() - start_time
        avg_time_per_row = elapsed_time / (index + 1)
        remaining_time = avg_time_per_row * (total_rows - (index + 1))
        pbar.set_postfix({"ETA (sec)": f"{remaining_time:.2f}"})

        print("\nOriginal Sentence:", row["text"])
        print("Tagged Sentence:", tagged)
        print("Label:", label)
        print("---")

Processing:  16%|█▋        | 53/323 [00:00<00:00, 525.81rows/s]

0  Skipped.
1  Skipped.
2  Skipped.
3  Skipped.
4  Skipped.
5  Skipped.
6  Skipped.
7  Skipped.
8  Skipped.
9  Skipped.
10  Skipped.
11  Skipped.
12  Skipped.
13  Skipped.
14  Skipped.
15  Skipped.
16  Skipped.
17  Skipped.
18  Skipped.
19  Skipped.
20  Skipped.
21  Skipped.
22  Skipped.
23  Skipped.
24  Skipped.
25  Skipped.
26  Skipped.
27  Skipped.
28  Skipped.
29  Skipped.
30  Skipped.
31  Skipped.
32  Skipped.
33  Skipped.
34  Skipped.
35  Skipped.
36  Skipped.
37  Skipped.
38  Skipped.
39  Skipped.
40  Skipped.
41  Skipped.
42  Skipped.
43  Skipped.
44  Skipped.
45  Skipped.
46  Skipped.
47  Skipped.
48  Skipped.
49  Skipped.
50  Skipped.
51  Skipped.
52  Skipped.
53  Skipped.
54  Skipped.
55  Skipped.
56  Skipped.
57  Skipped.
58  Skipped.
59  Skipped.
60  Skipped.
61  Skipped.
62  Skipped.
63  Skipped.
64  Skipped.
65  Skipped.
66  Skipped.
67  Skipped.
68  Skipped.
69  Skipped.
70  Skipped.
71  Skipped.
72  Skipped.
73  Skipped.
74  Skipped.
75  Skipped.
76  Skipped.
77  Skipp

Processing:  34%|███▍      | 110/323 [00:05<00:12, 16.76rows/s, ETA (sec)=10.94]


Original Sentence: I saw university students taking part in social movements and how universities are a place where there is the value of common governance by both teachers and students Lau Tsz-kei He said his political awakening came in 2012 , when he took part in a protest against compulsory national education for public schools , intended to strengthen “ national identity awareness ” and nurture patriotism towards China .
Tagged Sentence: I saw university students taking part in social movements and how universities are a place where there is the value of common governance by both teachers and students Lau Tsz-kei He said his political awakening came in 2012, when he took part in a protest against compulsory national education for public schools, intended to strengthen 'national identity awareness' and nurture patriotism towards China.
Label: 0
---


Processing:  34%|███▍      | 111/323 [00:18<00:12, 16.76rows/s, ETA (sec)=35.23]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|█████████▉| 322/323 [2:05:42<00:37, 37.63s/rows, ETA (sec)=23.42]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|██████████| 323/323 [2:06:03<00:00, 23.42s/rows, ETA (sec)=0.00] 


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the <effect>clashes</effect> are a desperate <trigger>protest</trigger> against <cause>religious oppression and economic marginalisation</cause>.
Label: 1
---


In [16]:
with open("test_pred_1.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [17]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [18]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [19]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [20]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [21]:
df_predictions

,tagged_sentence,label,original_sentence
0,The movement was catapulted into the headlines...,1,The movement was catapulted into the headlines...
1,Several thousand <effect>protesters took to th...,1,Several thousand protesters took to the street...
2,"“ They try every means to oppress us , ” compl...",1,"“ They try every means to oppress us , ” compl..."
3,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,NoTag,0,“ It is somehow like a revolution – no one wil...
...,...,...,...
318,NoTag,0,"Police spokesman , Lieutenant Colonel Thulani ..."
319,NoTag,0,"National Freedom Party spokesman , Sabelo Sigu..."
320,NoTag,0,He described the shooting as `` evil '' .
321,NoTag,0,"On Sunday , three explosions in the south of t..."


In [22]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [24]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8045977011494253
Accuracy: 0.7894736842105263
Precision: 0.8235294117647058
Recall: 0.7865168539325843
Matthews Correlation Coefficient: 0.5773875500645679


# F1 Score After 15 examples retrieved

In [8]:
total_rows = len(df)  
start_time = time.time()

with tqdm(total=total_rows, desc="Processing", unit="rows") as pbar:
    for index, row in df.iterrows():

        try:
            with open("test_pred_2.json", "r", encoding="utf-8") as f:
                data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            data = []

        flag = 0

        for i in range(len(data)):
            if data[i]["original"] == row["text"]:
                print(i, " Skipped.")
                flag = 1
                break

        if flag == 1:
            pbar.update(1)
            continue

        llm_output = generate(llm, row["text"], graph, embeddings)
        match = re.search(r"\{.*\}", llm_output, re.DOTALL)
        if match:
            json_data = match.group(0)
            try:
                json_data = json.loads(json_data)
            except json.JSONDecodeError:
                print(f"Error: Invalid JSON at row {index}. Skipping...")
                pbar.update(1)
                continue
        else:
            print(f"Error: No JSON found in row {index}. Skipping...")
            pbar.update(1)
            continue

        tagged = json_data.get("tagged", "")
        label = json_data.get("label", "")
        data.append({"tagged": tagged, "label": label, "original": row["text"]})

        with open("test_pred_2.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

        pbar.update(1)
        elapsed_time = time.time() - start_time
        avg_time_per_row = elapsed_time / (index + 1)
        remaining_time = avg_time_per_row * (total_rows - (index + 1))
        pbar.set_postfix({"ETA (sec)": f"{remaining_time:.2f}"})

        print("\nOriginal Sentence:", row["text"])
        print("Tagged Sentence:", tagged)
        print("Label:", label)
        print("---")

Processing:  47%|████▋     | 152/323 [00:00<00:00, 752.52rows/s]

0  Skipped.
1  Skipped.
2  Skipped.
3  Skipped.
4  Skipped.
5  Skipped.
6  Skipped.
7  Skipped.
8  Skipped.
9  Skipped.
10  Skipped.
11  Skipped.
12  Skipped.
13  Skipped.
14  Skipped.
15  Skipped.
16  Skipped.
17  Skipped.
18  Skipped.
19  Skipped.
20  Skipped.
21  Skipped.
22  Skipped.
23  Skipped.
24  Skipped.
25  Skipped.
26  Skipped.
27  Skipped.
28  Skipped.
29  Skipped.
30  Skipped.
31  Skipped.
32  Skipped.
33  Skipped.
34  Skipped.
35  Skipped.
36  Skipped.
37  Skipped.
38  Skipped.
39  Skipped.
40  Skipped.
41  Skipped.
42  Skipped.
43  Skipped.
44  Skipped.
45  Skipped.
46  Skipped.
47  Skipped.
48  Skipped.
49  Skipped.
50  Skipped.
51  Skipped.
52  Skipped.
53  Skipped.
54  Skipped.
55  Skipped.
56  Skipped.
57  Skipped.
58  Skipped.
59  Skipped.
60  Skipped.
61  Skipped.
62  Skipped.
63  Skipped.
64  Skipped.
65  Skipped.
66  Skipped.
67  Skipped.
68  Skipped.
69  Skipped.
70  Skipped.
71  Skipped.
72  Skipped.
73  Skipped.
74  Skipped.
75  Skipped.
76  Skipped.
77  Skipp

Processing:  53%|█████▎    | 172/323 [00:11<00:00, 752.52rows/s, ETA (sec)=10.02]


Original Sentence: State Home Minister R.R. Patil , who rushed to Gadchiroli , told reporters in the afternoon that the police have recovered the bodies of two Maoists gunned down in the fight .
Tagged Sentence: NoTag
Label: 0
---


Processing:  54%|█████▎    | 173/323 [00:36<00:48,  3.08rows/s, ETA (sec)=31.48] 

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|█████████▉| 322/323 [1:51:10<00:41, 41.67s/rows, ETA (sec)=20.71]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|██████████| 323/323 [1:51:57<00:00, 20.80s/rows, ETA (sec)=0.00] 


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the <effect>clashes</effect> are <trigger>a desperate protest</trigger> <cause>against religious oppression and economic marginalisation</cause>.
Label: 1
---


In [25]:
with open("test_pred_2.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [26]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [27]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [28]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [29]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [30]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [32]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8184281842818428
Accuracy: 0.7925696594427245
Precision: 0.7905759162303665
Recall: 0.848314606741573
Matthews Correlation Coefficient: 0.5792017135727426


# F1 Score after 20 examples

In [9]:
total_rows = len(df)  
start_time = time.time()

with tqdm(total=total_rows, desc="Processing", unit="rows") as pbar:
    for index, row in df.iterrows():

        try:
            with open("test_pred_3.json", "r", encoding="utf-8") as f:
                data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            data = []

        flag = 0

        for i in range(len(data)):
            if data[i]["original"] == row["text"]:
                print(i, " Skipped.")
                flag = 1
                break

        if flag == 1:
            pbar.update(1)
            continue

        llm_output = generate(llm, row["text"], graph, embeddings)
        match = re.search(r"\{.*\}", llm_output, re.DOTALL)
        if match:
            json_data = match.group(0)
            try:
                json_data = json.loads(json_data)
            except json.JSONDecodeError:
                print(f"Error: Invalid JSON at row {index}. Skipping...")
                pbar.update(1)
                continue
        else:
            print(f"Error: No JSON found in row {index}. Skipping...")
            pbar.update(1)
            continue

        tagged = json_data.get("tagged", "")
        label = json_data.get("label", "")
        data.append({"tagged": tagged, "label": label, "original": row["text"]})

        with open("test_pred_3.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

        pbar.update(1)
        elapsed_time = time.time() - start_time
        avg_time_per_row = elapsed_time / (index + 1)
        remaining_time = avg_time_per_row * (total_rows - (index + 1))
        pbar.set_postfix({"ETA (sec)": f"{remaining_time:.2f}"})

        print("\nOriginal Sentence:", row["text"])
        print("Tagged Sentence:", tagged)
        print("Label:", label)
        print("---")

Processing:   0%|          | 0/323 [00:00<?, ?rows/s]

0  Skipped.
1  Skipped.
2  Skipped.
3  Skipped.
4  Skipped.
5  Skipped.
6  Skipped.
7  Skipped.
8  Skipped.
9  Skipped.
10  Skipped.
11  Skipped.
12  Skipped.
13  Skipped.


Processing:  29%|██▉       | 95/323 [00:02<00:04, 50.67rows/s, ETA (sec)=49.52]


Original Sentence: Both sides were raining bombs on each other and Mondal was hit by one of the bombs , " Murshidabad district magistrate Pervez Ahmed Siddiqui said .
Tagged Sentence: NoTag
Label: 0
---
14  Skipped.
15  Skipped.
16  Skipped.
17  Skipped.
18  Skipped.
19  Skipped.
20  Skipped.
21  Skipped.
22  Skipped.
23  Skipped.
24  Skipped.
25  Skipped.
26  Skipped.
27  Skipped.
28  Skipped.
29  Skipped.
30  Skipped.
31  Skipped.
32  Skipped.
33  Skipped.
34  Skipped.
35  Skipped.
36  Skipped.
37  Skipped.
38  Skipped.
39  Skipped.
40  Skipped.
41  Skipped.
42  Skipped.
43  Skipped.
44  Skipped.
45  Skipped.
46  Skipped.
47  Skipped.
48  Skipped.
49  Skipped.
50  Skipped.
51  Skipped.
52  Skipped.
53  Skipped.
54  Skipped.
55  Skipped.
56  Skipped.
57  Skipped.
58  Skipped.
59  Skipped.
60  Skipped.
61  Skipped.
62  Skipped.
63  Skipped.
64  Skipped.
65  Skipped.
66  Skipped.
67  Skipped.
68  Skipped.
69  Skipped.
70  Skipped.
71  Skipped.
72  Skipped.
73  Skipped.
74  Skipped.
75 

Processing:  49%|████▊     | 157/323 [00:42<00:54,  3.07rows/s, ETA (sec)=44.57]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|█████████▉| 322/323 [1:31:39<00:49, 49.74s/rows, ETA (sec)=17.08]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , three <cause>explosions in the south of the region</cause> <trigger>killed</trigger> two people and <trigger>injured</trigger> many others .
Label: 1
---


Processing: 100%|██████████| 323/323 [1:32:33<00:00, 17.19s/rows, ETA (sec)=0.00] 


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the clashes are a <effect>desperate protest</effect> <trigger>against</trigger> <cause>religious oppression and economic marginalisation</cause> .
Label: 1
---


In [33]:
with open("test_pred_3.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [34]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [35]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [36]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [37]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [38]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [40]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8216216216216217
Accuracy: 0.7956656346749226
Precision: 0.7916666666666666
Recall: 0.8539325842696629
Matthews Correlation Coefficient: 0.5855831326919609
